In [2]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate

In [3]:
groq_api_key=os.getenv('GROQ_API_KEY')

In [4]:
llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="Llama3-8b-8192")

In [7]:
def sentiment_analyzer(input_text: str) -> list:
    template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a highly specialized AI trained in clinical psychology and mental health assessment. Your task is to analyze textual input and categorize it into one of the following mental health conditions:
    - Normal
    - Depression
    - Suicidal
    - Anxiety
    - Stress
    - Bi-Polar
    - Personality Disorder

    Your analysis should be based on clinical symptoms and diagnostic criteria commonly used in mental health practice. Here are some detailed examples:

    Example 1:
    Text: "I feel an overwhelming sense of sadness and hopelessness. I have lost interest in activities I once enjoyed and find it hard to get out of bed."
    Category: Depression

    Example 2:
    Text: "I constantly worry about various aspects of my life. My heart races, and I experience physical symptoms like sweating and trembling even when there is no apparent danger."
    Category: Anxiety

    Example 3:
    Text: "I have thoughts about ending my life. I feel that there is no other way to escape my pain, and I often think about how I might end it."
    Category: Suicidal

    Example 4:
    Text: "I feel extremely stressed and overwhelmed by my responsibilities. I find it difficult to relax, and I often experience headaches and tension."
    Category: Stress

    Example 5:
    Text: "I go through periods of extreme happiness and high energy, followed by episodes of deep depression and low energy. These mood swings affect my daily functioning."
    Category: Bi-Polar

    Example 6:
    Text: "I have trouble maintaining stable relationships and often experience intense emotional reactions. My self-image frequently changes, and I engage in impulsive behaviors."
    Category: Personality Disorder

    Example 7:
    Text: "I feel generally content and am able to manage my daily activities without significant distress or impairment."
    Category: Normal

    Now, analyze the following text and determine the most appropriate category from the list above, and return the Category and a brief explanation of your decision:
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Human: {input_text}
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    AI Assistant:"""

    sentiment_prompt = PromptTemplate(input_variables=["input_text"], template=template)
    initiator_router = sentiment_prompt | llm | StrOutputParser()
    output = initiator_router.invoke({"input_text":input_text})
    return output


In [8]:
sentiment = sentiment_analyzer("I feel incredibly anxious about everything and can't stop worrying")
print(sentiment)

Category: Anxiety

Explanation: The text indicates that the person is experiencing excessive and persistent worry, which is a hallmark symptom of anxiety disorder. The phrase "can't stop worrying" suggests that the individual is unable to control their worries, which is a common feature of anxiety disorders. Additionally, the phrase "anxious about everything" implies that the person is experiencing a pervasive and excessive anxiety that is interfering with their daily life. While anxiety can be a normal response to stressful situations, the severity and pervasiveness described in the text suggest that it may be a clinical concern.


In [12]:
sentiment = sentiment_analyzer("I feel like everything is falling apart around me. I'm constantly worried and can't seem to find any peace. My sleep is disturbed, and I often feel overwhelmed by even the smallest tasks.")
print(sentiment)

Category: Anxiety

My assessment is based on the following symptoms mentioned in the text:

* "Constantly worried" - This suggests that the individual is experiencing excessive and persistent worry, which is a hallmark symptom of anxiety.
* "Can't seem to find any peace" - This implies a sense of perpetual unease and inability to relax, which is also characteristic of anxiety.
* "Disturbed sleep" - Sleep disturbances are a common symptom of anxiety, often caused by racing thoughts and difficulty relaxing.
* "Overwhelmed by even the smallest tasks" - This suggests that the individual is experiencing feelings of excessive anxiety and difficulty coping with everyday activities, which is another common symptom of anxiety.

Overall, the text suggests that the individual is experiencing symptoms that are consistent with an anxiety disorder, such as generalized anxiety disorder or anxiety disorder not otherwise specified.


## Connecting to database

In [13]:
import os
from supabase import create_client, Client

url: str = os.environ.get("SUPABASE_PROJECT_URL")
key: str = os.environ.get("SUPABASE_API_KEY")
supabase: Client = create_client(url, key)

In [17]:
# signinh in with email and password
data = supabase.table("Interaction History").select("*").execute()

In [18]:
data

APIResponse[~_ReturnT](data=[], count=None)

In [20]:
new_row = {
    "Input_text" : "I feel incredibly anxious about everything and can't stop worrying",
    "Model_prediction" : "Anxiety",
    "Llama_3_Prediction" : "Anxiety",
    "Llama_3_Explanation" : "Anxiety",
    "User Rating" : 5,
}

data = supabase.table("Interaction History").insert(new_row).execute()

# Assert we pulled real data.
assert len(data.data) > 0

In [21]:
data

APIResponse[~_ReturnT](data=[{'id': 2, 'Input_text': "I feel incredibly anxious about everything and can't stop worrying", 'Model_prediction': 'Anxiety', 'Llama_3_Prediction': 'Anxiety', 'User Rating': 5, 'Llama_3_Explanation': 'Anxiety'}], count=None)